In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Данные константы
k = 0.011
c = 1.6
alpha = 0.5
l = 0.5 # длина цилиндра
radius = 4 # радиус
beta = 0.008
t = 180 # время наблюдения

In [6]:
# Задаем число отсчетов и шаги
num_m = 1000
num_i = 1000
tau = t/num_m # шаг по времени
h = radius/num_i # шаг по координате r
w = np.zeros((num_i, num_m))

def intensive(i): #зависимость
    assert i<num_i, "функция intensive(r) определена только для r>radius"
    r = i/num_i*radius
    if r == 0:
        return 5
    else:
        return 1/r

In [7]:
# краевое условие
w[:,0] = 0 # при t = 0

In [8]:
for m in range(0,num_m-1): # идем послойно по m
    w[0,m+1] = w[0,m] + tau/c*(4*k/h**2*(w[0,m]-w[1,m])-2*alpha/l*w[0,m]+beta/l*intensive(0))
    for i in range(1, num_i-2): # считаем каждый из i-ых элементов на слое m
        w[i,m+1] = w[i,m] + tau/c*(k/2/i/h**2*(w[i-1,m]-w[i+1,m])+
                                   k/h**2*(w[i-1,m]-2*w[i,m]-w[i+1,m])-2*alpha/l*w[i,m]+beta/l*intensive(i))
    w[num_i-1,m+1] = w[num_i-1,m] + tau/c*(2*k/h**2*(w[num_i-2,m]-w[num_i-1,m])
                                           -2*alpha/l*w[num_i-1,m]+beta/l*intensive(num_i-1))
    

/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
/home/kulinar/dlcourse_ai/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


In [11]:
def a(i):
    return k/(h**2)*(1/2/i + 1)
def b(i):
    return 2*c/h + 2*k/(h**2) + 2*alpha/l
def c(i):
    return -k/(2*i*h**2) - k/(h**2)